### ▶2023/08/11 杉井さんの先行研究のSOMを再現(参考 ../sugiisan/code/som_auto.ipynb)

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import somoclu
import pandas as pd
import seaborn
import shutil
import os
import glob
import re
import io

If you installed Somoclu with pip on Windows, this typically means missing DLLs. Please refer to the documentation.


In [3]:
n_rows, n_columns = 100, 100
folda_name = 'paper_bag_result'
os.mkdir(f'{folda_name}')
columns = ['bmu_Line', 'bmu_Column']
colorlist = ["red", "blue", "green", "yellow", "black"]

In [ ]:
gcnt = 0
fig = plt.figure()

for user in df_list :
    
    user_temp = re.split('[._/]', user)
    name = user_temp[3]
    time = user_temp[5]
    df = pd.read_table(user)
    cnt = 0
    labels = []
    
    if gcnt == 0 :
        os.mkdir(f'{folda_name}/{name}')
    os.mkdir(f'{folda_name}/{name}/{name}_{time}')
    
    for index, row in df.iterrows():
        
        random_cbook = 2 * np.random.rand(100, 100, 1886) - 1
        random_cbook = random_cbook.astype(np.float32)

        temp_x = row.iloc[1]
        temp_y = row.iloc[2]

        senshu = row.iloc[3]
        labels.append(str(senshu))

        array_x = [float(x) for x in temp_x.split(',')]
        array_y = [float(y) for y in temp_y.split(',')]
        
        if cnt == 0:
            
            array_xy = np.hstack((array_x, array_y))
        else:
            
            array_xy2 = np.hstack((array_x, array_y))
            array_xy = np.vstack((array_xy, array_xy2))
        cnt = cnt + 1
        
    array_xy = array_xy.astype(np.float32)
    print('fin make array')
    
    if gcnt == 0 :
        
        som = somoclu.Somoclu(n_columns, n_rows, initialcodebook = random_cbook, compactsupport=False)
        som.train(array_xy)
        gcnt = gcnt + 1
        
        print('fin som 1')
        
        codebook = som.codebook
        umatrix = som.umatrix
        inner = som.get_surface_state()
        best_match = som.get_bmus(inner)
        
        np.save(f'{folda_name}/{name}_{time}/codebook_{name}_{time}', codebook)
        np.savetxt(f'{folda_name}/{name}_{time}/umatrix_{name}_{time}.tsv', umatrix, delimiter='\t')
        np.savetxt(f'{folda_name}/{name}_{time}/bmu_{name}_{time}.tsv', best_match, delimiter='\t', fmt = '%d')
        
        initial_cbook = np.load(f'{folda_name}/{name}_{time}/codebook_{name}_{time}.npy')
        
        os.mkdir(f'som_strokes/{name}_{time}')
        
    else:
        
        som = somoclu.Somoclu(n_columns, n_rows, initialcodebook = initial_cbook, compactsupport=False)
        gcnt = gcnt + 1
        
        print('fin som 2')
        
        inner = som.get_surface_state(data = array_xy)
        best_match = som.get_bmus(inner)

        np.savetxt(f'{folda_name}/{name}_{time}/bmu_{name}_{time}.tsv', best_match, delimiter='\t', fmt = '%d')
        
        if gcnt == 3 :
            
            gcnt = 0
            
        else :
            
            pass
        
    df_2 = pd.DataFrame(best_match, index = None, columns = columns)
        
    df_3 = pd.concat([df, df_2], axis = 1)
    
    for index, row in df_3.iterrows():
        
        temp_x = int(row.iloc[4])
        temp_y = int(row.iloc[5])

        senshu = int(row.iloc[3])
        plt.scatter(temp_x, temp_y, c = colorlist[senshu], alpha = 0.5)
    plt.grid()
    plt.xlim(0, 100)
    plt.ylim(0, 100)
    plt.savefig(f'{folda_name}/{name}_{time}/fig_bmu_{name}_{time}.png')
    plt.cla()
    
    print(f'{name}_{time}_{gcnt}')